### 多群中子扩散方程求解程序
本程序为简易的多群中子扩散方程求解程序，为核反应堆物理课程实践。

参考《核反应堆物理分析》（谢仲生）第5版第5章5.3节 多群扩散方程的数值求解。

有待后续改进之处：
- 初始参数设置
- `update_phi`函数中`dx`和`dy`网格设置为非均匀，相应更新`k_eff`时做积分处理
- 根据收敛判断1和5调整初始值使得`k_eff`接近于1
- 函数封装
- numpy改为tensor，使用cuda加速



In [ ]:
import numpy as np
# 初始值设置
G=2
N=5
M=5
epsilon_1=1e-3
epsilon_3=1e-3
epsilon_5=1e-3
k_0=1.0
X=10.0
Y=10.0
dx=X/(N-1)
dy=Y/(M-1)
max_outer_iterations = 100

phi= [np.ones((N, M)) for g in range(G)]
Q= np.ones((N, M))
k_eff=1.0
S= [np.ones((N, M)) for g in range(G)]
chi=np.ones(G)
D_g = np.ones((N, M))
Sigma_r_g = np.ones((N, M))
nu_Sigma_f_g = np.ones(G)
Sigma_g_prime_g = np.ones((G, G))

# 收敛性判断函数
def judge_convergence_1(k_eff_new, k_eff_old, epsilon_1_input=epsilon_1):
        if k_eff_new == 0:
            return True
        return abs(1-k_eff_old/k_eff_new) < epsilon_1_input


def judge_convergence_3(phi_g_new, phi_g_old, epsilon_3_input=epsilon_3):
        diff_numerator = 0
        diff_denominator = 0
        diff_numerator = np.sum(np.abs(phi_g_new - phi_g_old))
        diff_denominator = np.sum(np.abs(phi_g_new))
        if diff_denominator == 0:  
            return True
        return (diff_numerator / diff_denominator) < epsilon_3_input

def judge_convergence_5(k_eff_new, k_0_input=k_0, epsilon_5_input=epsilon_5):
        return abs(k_eff_new-k_0_input) < epsilon_5_input

# 边界镜像索引函数
def mirror_index(index, max_index):
    if index < 0:
        return -index
    elif index >= max_index:
        return 2*max_index - index
    else:
        return index

def safe_get_index(array, i, j, N_input=N, M_input=M):
    i_safe = mirror_index(i, N_input)
    j_safe = mirror_index(j, M_input)
    return array[i_safe, j_safe]

def get_array_averages(array,i,j,N_input=N,M_input=M):
    array_SW = (safe_get_index(array, i-1, j-1, N_input, M_input)+safe_get_index(array, i, j-1, N_input, M_input)+safe_get_index(array, i, j, N_input, M_input)+safe_get_index(array, i-1, j, N_input, M_input))/4
    array_SE = (safe_get_index(array, i, j-1, N_input, M_input)+safe_get_index(array, i+1, j-1, N_input, M_input)+safe_get_index(array, i+1, j, N_input, M_input)+safe_get_index(array, i, j, N_input, M_input))/4
    array_NE = (safe_get_index(array, i, j, N_input, M_input)+safe_get_index(array, i+1, j, N_input, M_input)+safe_get_index(array, i+1, j+1, N_input, M_input)+safe_get_index(array, i, j+1, N_input, M_input))/4
    array_NW = (safe_get_index(array, i-1, j, N_input, M_input)+safe_get_index(array, i, j, N_input, M_input)+safe_get_index(array, i, j+1, N_input, M_input)+safe_get_index(array, i-1, j+1, N_input, M_input))/4
    return array_SW, array_SE, array_NE, array_NW

# 更新通量函数
def update_phi(phi_g_new,phi_g_old,S,g,D_g,Sigma_r_g,N_input=N,M_input=M,dx_input=dx,dy_input=dy):
    phi_g_temp = phi_g_new.copy()
    phi_g_old[0, :] = 0.0
    phi_g_old[-1, :] = 0.0
    phi_g_old[:, 0] = 0.0
    phi_g_old[:, -1] = 0.0

    phi_g_new[0, :] = 0.0
    phi_g_new[-1, :] = 0.0
    phi_g_new[:, 0] = 0.0
    phi_g_new[:, -1] = 0.0

    S_g = S[g]

    D_1 = np.zeros((N, M))
    D_2 = np.zeros((N, M))
    D_3 = np.zeros((N, M))
    D_4 = np.zeros((N, M))
    Sigma_r_1 = np.zeros((N, M))
    Sigma_r_2 = np.zeros((N, M))
    Sigma_r_3 = np.zeros((N, M))
    Sigma_r_4 = np.zeros((N, M))
    S_1 = np.zeros((N, M))
    S_2 = np.zeros((N, M))
    S_3 = np.zeros((N, M))
    S_4 = np.zeros((N, M))

    a_g = np.ones((N, M))
    b_g = np.ones((N, M))
    c_g = np.ones((N, M))
    d_g = np.ones((N, M))
    e_g = np.ones((N, M))

    for i in range(N):
            for j in range(M):
                 D_1[i,j],D_2[i,j],D_3[i,j],D_4[i,j] = get_array_averages(D_g,i,j,N_input,M_input)
                 Sigma_r_1[i,j],Sigma_r_2[i,j],Sigma_r_3[i,j],Sigma_r_4[i,j] = get_array_averages(Sigma_r_g,i,j,N_input,M_input)
                 S_1[i,j],S_2[i,j],S_3[i,j],S_4[i,j] = get_array_averages(S_g,i,j,N_input,M_input)
    for i in range(1,N):
            for j in range(1,M):

                a_g[i,j] = -(D_1[i,j]*dx+D_2[i,j]*dx)/(2*dy)
                b_g[i,j] = -(D_1[i,j]*dy+D_4[i,j]*dy)/(2*dx)
                c_g[i-1,j] = -(D_1[i-1,j]*dy+D_4[i-1,j]*dy)/(2*dx)
                d_g[i,j-1] = -(D_1[i,j-1]*dx+D_2[i,j-1]*dx)/(2*dy)
                e_g[i,j] = (Sigma_r_1[i,j]*dx*dy+Sigma_r_2[i,j]*dx*dy+Sigma_r_3[i,j]*dx*dy+Sigma_r_4[i,j]*dx*dy)/4-a_g[i,j]-b_g[i,j]-c_g[i-1,j]-d_g[i,j-1]
                S_g[i,j] = (S_1[i,j]*dx*dy+S_2[i,j]*dx*dy+S_3[i,j]*dx*dy+S_4[i,j]*dx*dy)/4


    for i in range(1,N):
            for j in range(1,M):
                phi_g_new[i,j] = -(a_g[i,j]*phi_g_new[i,j-1]+b_g[i,j]*phi_g_new[i-1,j]+c_g[i,j]*phi_g_old[i+1,j]+d_g[i,j]*phi_g_old[i,j+1]-S_g[i,j]) / e_g[i,j]
    phi_g_old = phi_g_temp.copy()
    return phi_g_new, phi_g_old, S

# 内迭代函数
def inner_iteration(phi,S,D_g,Sigma_r_g, epsilon_3_input=epsilon_3,G_input=G,N_input=N,M_input=M,dx_input=dx,dy_input=dy):
    for g in range(G_input):
        phi_g_new = phi[g]
        phi_g_old = phi[g].copy()
        while not judge_convergence_3(phi_g_new, phi_g_old, epsilon_3_input):
            phi_g_new, phi_g_old, S = update_phi(phi_g_new,phi_g_old,S,g,D_g,Sigma_r_g,N_input,M_input,dx_input,dy_input)
        phi[g] = phi_g_new
    return phi

# 更新中子源项和有效增殖因子函数
def update_Q_and_k(phi,Q_new, Q_old, k_eff_new, k_eff_old, nu_Sigma_f_g, G_input=G):
    Q_old = Q_new.copy()
    k_eff_old = k_eff_new
    Q_new = np.zeros_like(Q_old)
    for g_prime in range(G_input):
        Q_new += nu_Sigma_f_g[g_prime] * phi[g_prime]
    k_eff_new = k_eff_old*np.sum(Q_new)/np.sum(Q_old)
    return Q_new, Q_old, k_eff_new, k_eff_old

# 更新中子源项函数
def update_S(phi,Q,k_eff,S,Sigma_g_prime_g,chi, G_input=G):
    for g in range(G_input):
        S_g = np.sum([Sigma_g_prime_g[g_prime, g]*phi[g_prime] for g_prime in range(G_input)]) + chi[g]*Q/k_eff
        S[g] = S_g
    return S
     
     
# 外迭代函数
def outer_iteration(max_outer_iterations, phi,S, Q, k_eff, D_g, Sigma_r_g, nu_Sigma_f_g,chi,Sigma_g_prime_g, epsilon_1_input=epsilon_1, epsilon_5_input=epsilon_5, G_input=G):
    
    Q_new = Q.copy()
    Q_old = Q.copy()
    k_eff_new = k_eff
    k_eff_old = k_eff

    for n in range(max_outer_iterations):
        phi = inner_iteration(phi,S, D_g, Sigma_r_g, epsilon_3_input=epsilon_3, G_input=G, N_input=N, M_input=M, dx_input=dx, dy_input=dy)
        Q_new, Q_old, k_eff_new, k_eff_old = update_Q_and_k(phi,Q_new, Q_old, k_eff_new, k_eff_old, nu_Sigma_f_g, G_input=G)
        if judge_convergence_1(k_eff_new, k_eff_old, epsilon_1_input) and judge_convergence_5(k_eff_new, k_0_input=k_0, epsilon_5_input=epsilon_5):
            break
        S = update_S(phi,Q_new,k_eff_new,S, Sigma_g_prime_g, chi, G_input=G)

    return phi, Q_new, k_eff_new, S


In [ ]:

# 调用函数
phi_result, Q_result, k_eff_result, S_result = outer_iteration(
    max_outer_iterations=max_outer_iterations,
    phi=phi,          
    S=S,               
    Q=Q,              
    k_eff=k_eff,      
    D_g=D_g,           
    Sigma_r_g=Sigma_r_g, 
    nu_Sigma_f_g=nu_Sigma_f_g, 
    chi=chi,           
    Sigma_g_prime_g=Sigma_g_prime_g  
)

# 输出结果
print(f"最终 k_eff = {k_eff_result}")
print(f"最终 Q 的形状: {Q_result.shape}")
print(f"最终 phi 的能群数: {len(phi_result)}")
for g in range(G):
    print(f"能群 {g} 的通量范围: [{phi_result[g].min():.6f}, {phi_result[g].max():.6f}]")

for g in range(G):
    print(f"能群 {g+1} 的中子通量分布")
    print(phi_result[g])

最终 k_eff = 2.0
最终 Q 的形状: (5, 5)
最终 phi 的能群数: 2
能群 0 的通量范围: [1.000000, 1.000000]
能群 1 的通量范围: [1.000000, 1.000000]
能群 1 的中子通量分布
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
能群 2 的中子通量分布
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
